## 資料準備
* 先到  https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream.xml.bz2 下載
* 上傳到google drive上
* 在colab中掛載你的google drive

In [ ]:
# ! wget https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream.xml.bz2


In [ ]:
!wget https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2

In [ ]:
!pip install opencc-python-reimplemented

In [ ]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [ ]:
wiki_corpus

In [ ]:
next(iter(wiki_corpus.get_texts()))[:10]

In [ ]:
text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

In [ ]:
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)

In [ ]:
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
vector_size = 100
min_count = 1
workers = 8
epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')

In [ ]:
from gensim.models import word2vec

string = '電腦'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)